In [1]:
%run base_test.ipynb

import numpy as np

from base import ml
from base import plot

In [2]:
gen_one_data = ml.GenSolutionOnGrid(num_fenics=0)

In [3]:
dir_ = '../saved_models/grid_general'
dnn_factories, names = ml.dnn_factories_from_dir(dir_)
extra_facs = []
extra_names = []

# average_reg = ml.RegressorAverager(regressor_factories=dnn_factories)
# average_reg.auto_mask(gen_one_data=ml.gen_peakons_on_grid, thresh=0.5, batch_size=1000)
# average_reg_fac = ml.RegressorFactory(regressor=average_reg)
# extra_facs.append(average_reg_fac)
# extra_names.append('AverageReg')

# Warning: the polynomial interpolation takes a while.
# poly_deg = 10
# poly_fac = ml.RegressorFactory(regressor=ml.FineGridPolyInterp(poly_deg=poly_deg))
# bilin_fac = ml.RegressorFactory(regressor=ml.FineGridBilinearInterp())
# extra_facs.extend([poly_fac, bilin_fac])
# extra_names.extend(['Poly{}Reg'.format(poly_deg), 'BilinearReg'])

results = ml.eval_regressors([*dnn_factories,
                              *extra_facs], 
                             gen_one_data, 
                             10000)

bests = []
for stat in ('loss', 'max_deviation'):
    print(stat)
    print('-' * len(stat))
    results_names = sorted(zip(results, [*names, *extra_names]), 
                               key=lambda x: x[0][stat])
    bests.append([name for _, (result, name) in zip(range(6), results_names)])
    for result, name in results_names:
        print('{:>4} | {:>29} | {}'.format(eval(name.split('_')[0].replace('x', '*')), 
                                           name, 
                                           result[stat]))
    print('')
    
best = [x for x in bests[0] for best_ in bests[1:] if x in best_]
print('best')
print('----')
for b in best:
    print(b)

INFO:tensorflow:Using config: {'_model_dir': '../saved_models/grid_general/128x10_121__ScaleCenter_crelu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc1570df7f0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../saved_models/grid_general/128x10_121__ScaleCenter_crelu/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO: